In [ ]:
import geopandas
import rioxarray
import shapely
import pdal
import numpy
import json
import pathlib
import pdal
import time
import matplotlib
import matplotlib.pyplot

# The NZ20_Westport - fix the 15m offset when just using a Horizontal datum
Dataset information: https://doi.org/10.5069/G9Z31WV0

Datum information:

    Horizontal: NZTM2000 NZGD2000 Meters [EPSG: 2193]
    Vertical: NZVD2016 [EPSG: 7839]

In [ ]:
tile = "CL2_BR20_2020_1000_3612.laz"
file_path = pathlib.Path(r'C:\Users\pearsonra\Documents\data\Westport\local_cache\NZ20_Westport') / tile

In [ ]:
h_crs = 2193
v_crs = 7839

# NZ20_Westport - Load in with no in_srs set

In [ ]:
pdal_pipeline_instructions = [{"type":"readers.las", "filename": str(file_path)},
                              {"type":"filters.reprojection","out_srs":f"EPSG:{h_crs}"}, 
                              {"type":"filters.hexbin"}]
pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute()
metadata_hproj=json.loads(pdal_pipeline.get_metadata())
extent_hproj = shapely.wkt.loads(metadata_hproj['metadata']['filters.hexbin']['boundary'])
points_hproj = pdal_pipeline.arrays

In [ ]:
pdal_pipeline_instructions = [{"type":"readers.las", "filename": str(file_path)},
                              {"type":"filters.reprojection","out_srs":f"EPSG:{v_crs}"}, 
                              {"type":"filters.hexbin"}]
pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute()
metadata_vproj=json.loads(pdal_pipeline.get_metadata())
extent_vproj = shapely.wkt.loads(metadata_vproj['metadata']['filters.hexbin']['boundary'])
points_vproj = pdal_pipeline.arrays

In [ ]:
pdal_pipeline_instructions = [{"type":"readers.las", "filename": str(file_path)},
                              {"type":"filters.reprojection","out_srs":f"EPSG:{h_crs}+{v_crs}"}, 
                              {"type":"filters.hexbin"}]
pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute()
metadata_hvproj=json.loads(pdal_pipeline.get_metadata())
extent_hvproj = shapely.wkt.loads(metadata_vproj['metadata']['filters.hexbin']['boundary'])
points_hvproj = pdal_pipeline.arrays

In [ ]:
json.dumps(pdal_pipeline_instructions)

In [ ]:
pdal_pipeline_instructions = [{"type":"readers.las", "filename": str(file_path)},
                              {"type":"filters.hexbin"}]
pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute()
metadata_noproj=json.loads(pdal_pipeline.get_metadata())
extent_noproj = shapely.wkt.loads(metadata_noproj['metadata']['filters.hexbin']['boundary'])
points_noproj = pdal_pipeline.arrays

In [ ]:
ax = geopandas.GeoSeries(extent_hproj).plot(color='blue')
geopandas.GeoSeries(extent_vproj).plot(ax=ax, color='none', edgecolor='red', linewidth=4)
geopandas.GeoSeries(extent_noproj).plot(ax=ax, color='none', edgecolor='green', linewidth=2)


In [ ]:
metadata_hproj['metadata'].keys()

# NZ20_Westport - Compare the points returned for the different reprojections
Note that: 
1. The EPSG:2193 (horizontal) produces an incorrect vertical offset
2. The EPSG:7839 (vertical) results in no points returned
3. The EPSG:2193+7839 (horizontal + vertical) gives the correct results
4. No projection seems to produce correct value

**Check to see what happens when the vetical datum is not given as in Wellington_2013**. Consider chaining and checking projects and only performing projection if not already in the desired EPSG. Will need to apply clipping after as will need to check the points are in the right datum.

In [ ]:
points_hproj

In [ ]:
points_vproj

In [ ]:
points_hvproj

In [ ]:
points_noproj

# NZ20_Westport - Look at meta data to look for CRS
Note the `comp_spatialreference` and `spatialreference` are the same. The `srs` appears to indicate the transform applied. Could look at carefully to try understand the vertical offset.

In [ ]:
metadata_hproj['metadata'].keys()

In [ ]:
metadata_hproj['metadata']['filters.reprojection'].keys()

In [ ]:
metadata_hproj['metadata']['filters.reprojection']['comp_spatialreference'] == metadata_hproj['metadata']['filters.reprojection']['spatialreference']

In [ ]:
metadata_hproj['metadata']['filters.reprojection']['spatialreference']

In [ ]:
metadata_hproj['metadata']['filters.reprojection']['srs'] == metadata_hproj['metadata']['filters.reprojection']['spatialreference']

In [ ]:
metadata_hproj['metadata']['filters.reprojection']['srs'].keys()

In [ ]:
metadata_hproj['metadata']['filters.reprojection']['srs']

# NZ20_Westport - Look at Metadata to identify horizontal and vertical information

In [ ]:
metadata_noproj['metadata']['readers.las'].keys()

In [ ]:
metadata_noproj['metadata']['readers.las']['spatialreference']

In [ ]:
metadata_noproj['metadata']['readers.las']['spatialreference'] == metadata_noproj['metadata']['readers.las']['comp_spatialreference']

In [ ]:
metadata_noproj['metadata']['readers.las']['srs'].keys()

In [ ]:
metadata_noproj['metadata']['readers.las']['srs']['horizontal']

In [ ]:
metadata_noproj['metadata']['readers.las']['srs']['vertical']

In [ ]:
vertical_srs = metadata_noproj['metadata']['readers.las']['srs']['vertical']

In [ ]:
vertical_srs[vertical_srs.rfind('AUTHORITY["EPSG",'):].strip('AUTHORITY["EPSG",""').strip('"]]')

In [ ]:
metadata_noproj['metadata']['readers.las']['srs']['wkt']

# NZ20_Westport - Work around - Try reprojection again in_srs and out_srs both specified as the horizontal CRS
No longer needed as can specify both horizontal and vertical

Examples of pitfalls: https://pdal.io/workshop/exercises/translation/reprojection.html
1. This produces the correct z point values.
2. The Z SRS is lost. 

In [ ]:
pdal_pipeline_instructions = [{"type":"readers.las", "filename": str(file_path)},
                              {"type":"filters.reprojection","in_srs":"EPSG:" + str(h_crs), "out_srs":"EPSG:" + str(h_crs)}, 
                              {"type":"filters.hexbin"}]
pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute()
metadata_hproj_in_out = json.loads(pdal_pipeline.get_metadata())
extent_hproj_in_out = shapely.wkt.loads(metadata_hproj_in_out['metadata']['filters.hexbin']['boundary'])
points_hproj_in_out = pdal_pipeline.arrays

In [ ]:
points_hproj_in_out

In [ ]:
metadata_hproj_in_out['metadata']['readers.las']['srs']['horizontal']

In [ ]:
metadata_hproj_in_out['metadata']['readers.las']['srs']['vertical']

In [ ]:
metadata_hproj_in_out['metadata']['filters.reprojection']['comp_spatialreference'] == metadata_hproj_in_out['metadata']['filters.reprojection']['spatialreference']

In [ ]:
metadata_hproj_in_out['metadata']['filters.reprojection']['spatialreference']

In [ ]:
metadata_hproj_in_out['metadata']['filters.reprojection']['srs']['vertical']

# NZ20_Westport - Work around - Try reprojection again with in_srs and out_srs both specified as the horizontal + vertical one
No longer needed as can specify both horizontal and vertical

What happens if we use the combined SRS in our projection for the in and out
1. This produces the correct z point values.
2. The vertical SRS appears to be lost. 

In [ ]:
pdal_pipeline_instructions = [{"type":"readers.las", "filename": str(file_path)},
                              {"type":"filters.reprojection",
                               "in_srs": metadata_noproj['metadata']['readers.las']['spatialreference'], 
                               "out_srs": metadata_noproj['metadata']['readers.las']['spatialreference']}, 
                              {"type":"filters.hexbin"}]
pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute()
metadata_hvproj_in_out = json.loads(pdal_pipeline.get_metadata())
extent_hvproj = shapely.wkt.loads(metadata_hvproj_in_out['metadata']['filters.hexbin']['boundary'])
points_hvproj = pdal_pipeline.arrays

In [ ]:
points_hproj

In [ ]:
metadata_hproj_in_out['metadata']['filters.reprojection']['srs']['vertical']

In [ ]:
metadata_hproj_in_out['metadata']['filters.reprojection']['srs']['horizontal']

# Wellington_2013 - no vertical secified
Vertical given as NZVD2016, but no EPSG code is given for the vertical datum: https://portal.opentopography.org/datasetMetadata?otCollectionID=OT.042017.2193.2

What happens if we specify a vertical component to the reprojection?

In [ ]:
tile = "ot_CL1_WLG_2013_1km_074035.laz"
file_path = pathlib.Path(r'C:\Users\pearsonra\Documents\data\Wakanae\local_cache\Wellington_2013') / tile

In [ ]:
pdal_pipeline_instructions = [{"type":"readers.las", "filename": str(file_path)},
                              {"type":"filters.reprojection","out_srs":f"EPSG:{h_crs}"}, 
                              {"type":"filters.hexbin"}]
pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute()
metadata_hproj=json.loads(pdal_pipeline.get_metadata())
extent_hproj = shapely.wkt.loads(metadata_hproj['metadata']['filters.hexbin']['boundary'])
points_hproj = pdal_pipeline.arrays

In [ ]:
pdal_pipeline_instructions = [{"type":"readers.las", "filename": str(file_path)},
                              {"type":"filters.reprojection","out_srs":f"EPSG:{h_crs}+{v_crs}"}, 
                              {"type":"filters.hexbin"}]
pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute()
metadata_hvproj=json.loads(pdal_pipeline.get_metadata())
extent_hvproj = shapely.wkt.loads(metadata_hproj['metadata']['filters.hexbin']['boundary'])
points_hvproj = pdal_pipeline.arrays

In [ ]:
pdal_pipeline_instructions = [{"type":"readers.las", "filename": str(file_path)}, 
                              {"type":"filters.reprojection","in_srs":f"EPSG:{h_crs}+{v_crs}",
                               "out_srs":f"EPSG:{h_crs}+{v_crs}"},
                              {"type":"filters.hexbin"}]
pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute()
metadata_noproj=json.loads(pdal_pipeline.get_metadata())
extent_noproj = shapely.wkt.loads(metadata_hproj['metadata']['filters.hexbin']['boundary'])
points_noproj = pdal_pipeline.arrays

## Plot out the 1st five points of each
Are the horizontal and horizontal+vertical points the same?

In [ ]:
points_hproj[0:5]

In [ ]:
points_hvproj[0:5]

In [ ]:
points_noproj[0:5]

**No they are not.** This means we need a way of setting the projections if they are not set previously!

# Example code for Martin - PhD rotations

In [ ]:
import pdal
import numpy
import json
import pathlib

h_crs=2193
v_crs=7839

# read in the file
pdal_pipeline_instructions = [{"type":"readers.las", "filename": str(file_path)}, 
                              {"type":"filters.reprojection","in_srs":f"EPSG:{h_crs}+{v_crs}",
                               "out_srs":f"EPSG:{h_crs}+{v_crs}"}]
pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute()
metadata=json.loads(pdal_pipeline.get_metadata())
points = pdal_pipeline.arrays # note is shape [1,nx,ny]

# do your rotation here
#### your code

# overwrite the X, Y, and Z values in the points array with the rotated values
points[0]['X'] = rotated_x
points[0]['Y'] = rotated_y
points[0]['Z'] = rotated_x

# save out the file
pdal_pipeline_instructions = [
    {"type":  "writers.las",
     "a_srs": f"EPSG:{h_crs}+{v_crs}",
     "filename": str(lidar_file),
     "compression": "laszip"}
]

pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions), [points])
pdal_pipeline.execute()